In [1]:
import numpy as np
import cv2
from PIL import Image
import matplotlib.pyplot as plt
from scipy import linalg

In [2]:
def loadVid(path):
	# Create a VideoCapture object and read from input file
	# If the input is the camera, pass 0 instead of the video file name
	cap = cv2.VideoCapture(path)

	# Check if camera opened successfully
	if (cap.isOpened()== False):
		print("Error opening video stream or file")

	i = 0
	# Read until video is completed
	while(cap.isOpened()):
		# Capture frame-by-frame
		i += 1
		ret, frame = cap.read()
		if ret == True:

			#Store the resulting frame
			if i == 1:
				frames = frame[np.newaxis, ...]
			else:
				frame = frame[np.newaxis, ...]
				frames = np.vstack([frames, frame])
				frames = np.squeeze(frames)

		else:
			break

	# When everything done, release the video capture object
	cap.release()

	return frames

In [3]:
img_reference = cv2.imread("cv_cover.jpg",0)

In [4]:
frames=loadVid("Part 1/book.mov")
panda_frames=loadVid("Part 1/ar_source.mov")

In [1]:
def form_A_mat(corres_ref,corres_frame):
	A = np.empty((0,9))
	for i in range(len(corres_ref)):
		u,v=corres_frame[i]
		x,y=corres_ref[i]
		x=np.array([
				   [-x , -y , -1 , 0 ,0 ,0 ,u*x, u*y , u],
				   [0,0,0,-x,-y,-1,v*x,v*y,v]
				   ])
		A = np.vstack( (A,x) )
	return A

In [7]:
def overlay(img_reference,book_frame,panda_frame,segma=0.4):

	sift = cv2.SIFT_create()
	kp_reference, des_reference = sift.detectAndCompute(img_reference,None)  #GET SIFT DESCRIPTORS AND KEYPOINTS FOR COVER

	kp_frame, des_frame = sift.detectAndCompute(book_frame,None)

	#MATCH DESCRIPTORS IN COVER AND BOOK FRAME
	bf = cv2.BFMatcher()
	matches = bf.knnMatch(des_reference,des_frame, k=2)

	#FILTER GOOD CORRESPONDANCES AND THEIER COORDINATES
	good = []
	correspondence_reference = []
	correspondence_frame = []
	for m,n in matches:
		if m.distance < segma*n.distance:
			good.append([m])
			correspondence_reference.append(kp_reference[m.queryIdx].pt)
			correspondence_frame.append(kp_frame[m.trainIdx].pt)

	good=good[:50]	#TO PLOT MATCHES

	#GET A TO SOLVE DLT AH=0
	A=form_A_mat(correspondence_reference,correspondence_frame)
	u, s, v = linalg.svd(A)
	DOF=np.reshape(v[-1],(3,3))


	ur_pixel=np.matmul(DOF,(img_reference.shape[1],0,1))/np.matmul(DOF,(img_reference.shape[1],0,1))[2]
	ul_pixel=np.matmul(DOF,(0,0,1))/np.matmul(DOF,(0,0,1))[2]
	lr_pixel=np.matmul(DOF,(img_reference.shape[1],img_reference.shape[0],1))/np.matmul(DOF,(img_reference.shape[1],img_reference.shape[0],1))[2]
	ll_pixel=np.matmul(DOF,(0,img_reference.shape[0],1))/np.matmul(DOF,(0,img_reference.shape[0],1))[2]

	#RESUZIE ROWS
	col_to_crop=int((panda_frame.shape[1]-img_reference.shape[1])/2)
	xx=panda_frame[ : , col_to_crop:panda_frame.shape[1]-col_to_crop, :]
	xx=cv2.resize(xx, (350,440))

	temp_frame=book_frame.copy()
	for j in range (xx.shape[0]):
		for i in range (xx.shape[1]):
			try:
				res=np.matmul(DOF,(i,j, 1))
				res=np.ceil(res/res[2]).astype(int)
				temp_frame[res[1],res[0],:]=xx[j,i,]
			except:
				print("hi")
				return overlay(img_reference,book_frame,panda_frame,segma=segma+0.1)

	return temp_frame

In [10]:
import matplotlib as mpl
mpl.rc('figure', max_open_warning = 0)
for i in range(len(panda_frames)):
	img=overlay(img_reference,frames[i].copy(),panda_frames[i].copy(),segma=0.4)
	cv2.imwrite("saved/img_"+str(i)+".jpg", img)
